<a href="https://colab.research.google.com/github/yhp2205/SQL/blob/main/ch_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Colab에서 Bigquery 사용해보기
bigquery를 사용하기 위해서 구글 드라이브에 권한을 허용해줍니다.

실행 시 뜨는 링크를 통해 계정을 로그인하면 나오는 코드를 빈칸에 입력합니다.

In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 5.1 코드를 레이블로 변경하는 쿼리
코드를 레이블로 변경하는 쿼리를 작성합니다. 

bigquery에서 project를 생성하고, dataset을 미리 만들어놓습니다.  
project id : mygcpproject-340112,  
dataset name = sqldata  
로 지정했을 때의 코드 작성입니다.   
데이터 셋의 이름은 sqldata, 테이블의 이름은 mst_users로 테이블을 만듭니다.

In [2]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.mst_users;
CREATE TABLE sqldata.mst_users(
    user_id         STRING
  , register_date   STRING
  , register_device NUMERIC
);

""


앞서 user_id, register_date, register_device 변수 생성과 형식을 지정해주었습니다.  
그리고 이어서 Value값을 테이블에 넣어주는 코드를 작성합니다.

In [3]:
%%bigquery --project mygcpproject-340112
INSERT INTO sqldata.mst_users
VALUES
    ('U001', '2016-08-26', 1)
  , ('U002', '2016-08-26', 2)
  , ('U003', '2016-08-27', 3)
;

""


select 구문으로 USER_ID를 선택하고, CASE 구문을 사용하여 register_device 값에 따른 device_name 변수를 만들어 출력합니다. 

In [4]:
%%bigquery --project mygcpproject-340112
SELECT
	USER_ID
	, CASE
		WHEN register_device = 1 THEN '데스크톱'
		WHEN register_device = 2 THEN '스마트폰'
		WHEN register_device = 3 THEN '애플리케이션'
		-- 디폴트 값을 지정할 경우 ELSE 구문을 사용합니다.
		-- ELSE ''
	END AS device_name
FROM sqldata.mst_users
;

,USER_ID,device_name
0,U003,애플리케이션
1,U002,스마트폰
2,U001,데스크톱


### 5.2 referrer domain을 추출하는 쿼리
referrer domain을 추출하는 쿼리를 만들어보겠습니다. 

access_log라는 이름의 데이터 테이블을 만들어 보겠습니다. stamp는 datetime으로,  
 url이 들어갈 referrer과 url은 문자형 자료이기 때문에 STRING으로 지정해줍니다. 

In [5]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.access_log;
CREATE TABLE sqldata.access_log(
    stamp    DATETIME
  , referrer STRING
  , url      STRING
);

""


dataset에 데이터를 입력해줍니다. 

In [6]:
%%bigquery --project mygcpproject-340112
INSERT INTO sqldata.access_log 
VALUES
    ('2016-08-26 12:02:00', 'http://www.other.com/path1/index.php?k1=v1&k2=v2#Ref1', 'http://www.example.com/video/detail?id=001')
  , ('2016-08-26 12:02:01', 'http://www.other.net/path1/index.php?k1=v1&k2=v2#Ref1', 'http://www.example.com/video#ref'          )
  , ('2016-08-26 12:02:01', 'https://www.other.com/'                               , 'http://www.example.com/book/detail?id=002' )
;


""


referrer의 도메인 부분을 호출하는 쿼리를 작성합니다. 

In [7]:
%%bigquery --project mygcpproject-340112
SELECT
	stamp
	-- referrer의 호스트 이름 부분 추출하기
  , FORMAT("%T", NET.HOST(referrer)) AS referrer_host
FROM sqldata.access_log
;

,stamp,referrer_host
0,2016-08-26 12:02:00,"""www.other.com"""
1,2016-08-26 12:02:01,"""www.other.com"""
2,2016-08-26 12:02:01,"""www.other.net"""


### 5.3 URL 경로와 GET 매개변수에 있는 특정 키 값을 추출하는 쿼리
URL 경로와 GET 매개변수에 있는 특정한 키 값을 추출하는 쿼리를 작성해보겠습니다.   
데이터는 5.2에서 만든 테이블을 사용하겠습니다. 

url 경로에서 원하는 부분을 뽑아내야하기 때문에 빅쿼리 정규 표현식을 사용하겠습니다. REGEXP_EXTRACT 함수를 사용하여 url에서 뽑아낼 path와 id 변수의 value를 작성합니다. 

In [8]:
%%bigquery --project mygcpproject-340112
SELECT
	stamp
  , url
  , REGEXP_EXTRACT(url, '//[^/]+([^?#]+)') AS path
  , REGEXP_EXTRACT(url, 'id=([^&]*)') AS id
FROM sqldata.access_log
;

,stamp,url,path,id
0,2016-08-26 12:02:00,http://www.example.com/video/detail?id=001,/video/detail,001
1,2016-08-26 12:02:01,http://www.example.com/book/detail?id=002,/book/detail,002
2,2016-08-26 12:02:01,http://www.example.com/video#ref,/video,None


### 5.4 URL 경로를 슬래시로 분할하여 계층 추출하기
이번에는 URL 경로를 슬래시를 기준으로 분할하여 값을 추출해보는 쿼리를 작성해보겠습니다. 

In [10]:
%%bigquery --project mygcpproject-340112
SELECT
	stamp
  , url
  -- 경로를 슬래시로 잘라 배열로 분할하기
  -- url에서 경로가 반드시 슬래시로 시작하기 때문에 2번째 요소가 마지막 계층
  -- split를 사용하여 배열로 자르기
  , split(REGEXP_EXTRACT(url, '//[^/]+([^?#]+)'), '/')[SAFE_ORDINAL(2)] AS path1
  , split(REGEXP_EXTRACT(url, '//[^/]+([^?#]+)'), '/')[SAFE_ORDINAL(3)] AS path2
FROM sqldata.access_log
;

,stamp,url,path1,path2
0,2016-08-26 12:02:00,http://www.example.com/video/detail?id=001,video,detail
1,2016-08-26 12:02:01,http://www.example.com/book/detail?id=002,book,detail
2,2016-08-26 12:02:01,http://www.example.com/video#ref,video,None


### 5.5 현재 날짜와 타임 스탬프 추출하기
새로운 데이터 테이블을 사용하여 날짜와 타임스탬프를 추출하는 쿼리를 작성해보겠습니다. 

우선 테이블을 생성합니다. 

In [11]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.purchase_log_with_coupon;
CREATE TABLE sqldata.purchase_log_with_coupon (
    purchase_id STRING
  , amount      integer
  , coupon      integer
)
;

""


value 값을 입력합니다.

In [12]:
%%bigquery --project mygcpproject-340112
INSERT INTO sqldata.purchase_log_with_coupon
VALUES
    ('10001', 3280, NULL)
  , ('10002', 4650,  500)
  , ('10003', 3870, NULL)
;

""


이 테이블을 이용하여 날짜와 타임스탬프를 추출할 수 있습니다. 

In [13]:
%%bigquery --project mygcpproject-340112
SELECT
  --CURRENT_DATE() 함수와 CURRENT_TIMESTAMP() 함수 사용하기
  CURRENT_DATE() AS dt
  , CURRENT_TIMESTAMP() AS stamp
;

,dt,stamp
0,2022-02-04,2022-02-04 08:37:37.491679+00:00


### 5.6 문자열을 날짜 자료형, 타임스탬프 자료형으로 변환
문자열을 날짜나 타임스탬프 형식으로 변환하는 쿼리를 작성해보겠습니다. 

In [14]:
%%bigquery --project mygcpproject-340112
SELECT
  -- 'CAST(value AS type)' 사용하기
  CAST('2016-01-30' AS date) AS dt
  , CAST('2016-01-30 12:00:00' AS timestamp) AS stamp
;

,dt,stamp
0,2016-01-30,2016-01-30 12:00:00+00:00
